# Attention으로 번역기 만들기
- Attention 

## 1. 패키지 import 하이퍼파라미터
- 하이퍼파라미터 : 모델의 정확도 및 학습속도에 영향을 미치는 변수

In [1]:
import numpy as np
import pandas as pd

from keras.models import Model
from keras.layers import Input, Dense, LSTM, Attention, Concatenate
from keras.utils import to_categorical

In [3]:
MY_HIDDEN = 128
MY_EPOCH = 500

## 2 번역 데이터 가져오기

In [2]:
raw = pd.read_csv('data/translate.csv', header=None)
eng_kor = raw.values.tolist()

## 3. 영어 알파벳과 한글문자 리스트 만들기

In [6]:
# 영어 알파벳 리스트
e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
# 한글문자 리스트
k_alpha = pd.read_csv('data/korean.csv', header=None)[0].tolist()

In [7]:
alpha = e_alpha + k_alpha
alpha_total_size = len(alpha)

## 4. 문자당 num를 갖는 dict 만들기

In [9]:
char_to_num = {c:i for i, c in enumerate(alpha)}
num_to_char = {i:c for i, c in enumerate(alpha)}

In [10]:
data = eng_kor[0]

## 5. 인코더 입력, 디코더 입력, 디코더 출력
- 인코더 입력 : 영어 알파벳 -> 숫자 -> 원핫인코딩(110,4,171)
- 디코더 입력 : 'S'+한글문자 -> 숫자 -> 원핫인코딩(110,4,171)
- 디코더 출력 : 한글문자+'E' -> 숫자(110, 3)인 list -> 110,3,1 배열로

In [11]:
def encoding(eng_kor = eng_kor):
    enc_in = []
    dec_in = []
    dec_out = []
    for data in eng_kor:
        # 인코더 입력데이터(영어알파벳 -> 숫자 _> 원핫인코딩)
        eng = [char_to_num[c] for c in data[0]]
        eng_one = np.eye(alpha_total_size)[eng]
        enc_in.append(eng_one)

        # 디코더 입력데이터('S'+한글 -> 숫자 -> 원핫인코딩)
        kor = [char_to_num[c] for c in 'S'+data[1]]
        kor_one = np.eye(alpha_total_size)[kor]
        dec_in.append(kor_one)
        
        # 디코더 출력데이터(한글+'E' -> 숫자)
        kor = [char_to_num[c] for c in data[1]+'E']
        dec_out.append(kor)

    return enc_in, dec_in, dec_out

## 6. 전체 입력데이터, 타겟데이터 준비

In [12]:
x_enc, x_dec, y_dec = encoding(eng_kor)
X_enc = np.array(x_enc)
X_dec = np.array(x_dec)
Y_dec = np.array(y_dec).reshape(-1,3,1)

X_enc.shape, X_dec.shape, Y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3, 1))

## 7 모델 구현(Seq2Seq)

In [ ]:
# 인코더 LSTM
ENC_IN = Input(shape=(4,alpha_total_size))

_, state_h, state_c = LSTM(units=MY_HIDDEN, return_state=True)(ENC_IN)

# 인코더와 디코더 연결 고리
LINK = [state_h, state_c]

# 디코더 LSTM
DEC_IN = Input(shape=(3,alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN, 
               return_sequences=True # 윗 출력 받음
               )(DEC_IN, initial_state=LINK)

DEC_OUT = Dense(units=alpha_total_size, activation='softmax')(DEC_MID)

# 모델
model = Model(inputs = [ENC_IN,DEC_IN],
              outputs = DEC_OUT)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 4, 171)]     0           []                               
                                                                                                  
 input_15 (InputLayer)          [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm_11 (LSTM)                 [(None, 128),        153600      ['input_14[0][0]']               
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

## 7. 모델구현(Attention)

In [16]:
# 인코더 입력
ENC_IN = Input(shape=(4,alpha_total_size))

# 인코드 LSTM : 모든 LSTM 스텝 출력
ENC_OUT, state_h, state_c = LSTM(units=MY_HIDDEN, 
                                 return_sequences= True,  # LSTM 모든 스텝의 윗 출력
                                 return_state=True        # h값과 c값 받기
                                 )(ENC_IN)

# 인코더와 디코더 연결 고리
LINK = [state_h, state_c]

# 디코더 LSTM
DEC_IN = Input(shape=(3,alpha_total_size))

# 디코더 LSTM
DEC_LSTM_OUT, _, _ = LSTM(units=MY_HIDDEN, 
                          return_sequences=True, 
                          return_state = True
                         )(DEC_IN, initial_state=LINK)

# Attention vector(매커니즘 정의)
CONTEXT_VECTOR = Attention()([DEC_LSTM_OUT, ENC_OUT])

# 컨텍스트벡터와 디코드 LSTM 결과를 concat
CONTEXT_AND_LSTM_OUT = Concatenate()([CONTEXT_VECTOR,DEC_LSTM_OUT])

#출력층
OUT = Dense(units=alpha_total_size, activation='softmax')(CONTEXT_AND_LSTM_OUT)

# 모델정의
model = Model(inputs=[ENC_IN, DEC_IN], outputs=OUT)

# 모델
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 4, 171)]     0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm_6 (LSTM)                  [(None, 4, 128),     153600      ['input_7[0][0]']                
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                            

## 8. 학습과정 설정 & 학습

In [17]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [18]:
hist = model.fit([X_enc, X_dec], Y_dec, epochs=MY_EPOCH)

Epoch 1/500
4/4 [==============================] - 4s 12ms/step - loss: 5.1173 - accuracy: 0.1909
Epoch 2/500
4/4 [==============================] - 0s 18ms/step - loss: 4.9617 - accuracy: 0.3333
Epoch 3/500
4/4 [==============================] - 0s 11ms/step - loss: 4.2538 - accuracy: 0.3333
Epoch 4/500
4/4 [==============================] - 0s 10ms/step - loss: 3.4623 - accuracy: 0.3333
Epoch 5/500
4/4 [==============================] - 0s 10ms/step - loss: 3.4008 - accuracy: 0.3333
Epoch 6/500
4/4 [==============================] - 0s 10ms/step - loss: 3.3616 - accuracy: 0.3333
Epoch 7/500
4/4 [==============================] - 0s 10ms/step - loss: 3.3264 - accuracy: 0.3333
Epoch 8/500
4/4 [==============================] - 0s 10ms/step - loss: 3.3008 - accuracy: 0.3333
Epoch 9/500
4/4 [==============================] - 0s 10ms/step - loss: 3.2819 - accuracy: 0.3333
Epoch 10/500
4/4 [==============================] - 0s 10ms/step - loss: 3.2503 - accuracy: 0.3333
Epoch 11/500
4/4 [=

## 9. 모델사용

In [19]:
# 쉬운문제
easy_test = [['loss','PP'],
             ['male','PP'],
             ['luck','PP'],
             ['milk','PP']]
enc_in, dec_in, _ =encoding(easy_test)
enc_in = np.array(enc_in)
dec_in = np.array(dec_in)
enc_in.shape, dec_in.shape

((4, 4, 171), (4, 3, 171))

In [20]:
# 위의 문제 예측하기
pred = model.predict([enc_in,dec_in])
pred = pred.argmax(axis=-1)
for test,yhat in zip(easy_test,pred):
    word = ''.join([num_to_char[char] for char in yhat[:-1]])
    print( test[0], ':',word)
   

1/1 [==============================] - 1s 840ms/step
loss : 손해
male : 남자
luck : 행운
milk : 우유


In [21]:
# 어려운 문제
hard_test = [['lsos','PP'],
             ['mlae','PP'],
             ['luck','PP'],
             ['mlik','PP']]
enc_in, dec_in, _ =encoding(hard_test)
enc_in = np.array(enc_in)
dec_in = np.array(dec_in)
pred = model.predict([enc_in,dec_in]).argmax(axis=-1)
for test,yhat in zip(hard_test,pred):
    word = ''.join([num_to_char[char] for char in yhat[:-1]])
    print( test[0], ':',word)

1/1 [==============================] - 0s 23ms/step
lsos : 손해
mlae : 남자
luck : 행운
mlik : 우유
